<a href="https://colab.research.google.com/github/mosovam/react-medical-view/blob/main/Model_to_ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/8c7f0be1e1c3803fcb4c41bcd9f4226b/super_resolution_with_onnxruntime.ipynb

https://pytorch.org/docs/master/onnx.html#example-alexnet-from-pytorch-to-onnx

In [ ]:
%pip install segmentation-models-pytorch

     |████████████████████████████████| 88 kB 3.4 MB/s 
     |████████████████████████████████| 376 kB 56.8 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=daf34d6c9e12efc201779b3bbc615ef47d24949dde2200222964f682162e097c
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=8fc8cf662b090c6dfb46501a3c0e99585765791f8402873db82695eb65316dc5
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
# Some standard imports
import io
import numpy as np
import segmentation_models_pytorch as smp

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

In [ ]:
# import the files from google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/OSU_IT/BC_MMO'
ENCODER = 'se_resnext50_32x4d'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'
CLASSES = ['tumor', 'unlabelled']

my_model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION
)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth


  0%|          | 0.00/105M [00:00<?, ?B/s]

In [ ]:
path = FOLDER_PATH + '/tumor_model/my_segmentation_model_saved_as_epochs8'

my_model.load_state_dict(torch.load(path))
my_model.eval()

Unet(
  (encoder): SENetEncoder(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
if torch.cuda.is_available():
    print('Model on cuda!')
    my_model.cuda()

Model on cuda!


In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 9            |        cudaMalloc retries: 9         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   10705 MB |   10705 MB |   19322 MB |    8617 MB |\n|       from large pool |   10658 MB |   10658 MB |   19151 MB |    8493 MB |\n|       from small pool |      47 MB |      47 MB |     171 MB |     124 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   10705 MB |   10705 MB |   19322 MB |    8617 MB |\n|       from large pool |   10658 MB |   10658 MB |

In [ ]:
# Export the model
dummy_input = torch.randn(1, 3, 512, 512, device="cuda")
torch.onnx.export(my_model,               # model being run
                  dummy_input,               # model input (or a tuple for multiple inputs)
                  FOLDER_PATH + "/brain_tumor_segmentation.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file,
                  opset_version=12,          # the ONNX version to export the model to
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK
)

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:269: UserWarning: `add_node_names' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `add_node_names` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:269: UserWarning: `do_constant_folding' can be set to True only when 'operator_export_type' is `ONNX`. Since 'operator_export_type' is not set to 'ONNX', `do_constant_folding` argument will be ignored.
  "`{}` argument will be ignored.".format(arg_name, arg_name))


In [ ]:
!pip install datasets git+https://github.com/neuml/txtai#egg=txtai[pipeline]

  Cloning https://github.com/neuml/txtai to /tmp/pip-install-0dbq_w1e/txtai_0d3c0cd5b4c94bac97d1540623a1f119
  Running command git clone -q https://github.com/neuml/txtai /tmp/pip-install-0dbq_w1e/txtai_0d3c0cd5b4c94bac97d1540623a1f119
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached datasets-2.1.0-py3-none-any.whl (325 kB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached faiss_cpu-1.7.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached ImageHash-

In [ ]:
import io
import numpy as np

from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx as onnx

In [ ]:
%pip install onnxruntime

In [ ]:
import onnxruntime

In [ ]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort /content/drive/MyDrive/OSU_IT/BC_MMO/brain_tumor_segmentation.onnx


Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /content/drive/MyDrive/OSU_IT/BC_MMO/brain_tumor_segmentation.onnx to ORT format model /content/drive/MyDrive/OSU_IT/BC_MMO/brain_tumor_segmentation.ort
2022-04-19 21:13:01.232040757 [W:onnxruntime:, graph.cc:1271 Graph] Initializer encoder.layer1.0.se_module.fc1.weight appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2022-04-19 21:13:01.232101182 [W:onnxruntime:, graph.cc:1271 Graph] Initializer encoder.layer1.0.se_module.fc1.bias appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph i